In [1]:
# main package
import random, time, numpy as np, vizdoom as vd

In [2]:
# loading the main object, this is the instance of the Doom game - https://vizdoom.farama.org/api/python/doomGame/
game = vd.DoomGame() 

# Loads configuration from a configuration file
game.load_config('resource/ViZDoom/scenarios/deadly_corridor.cfg')

# Initializes ViZDoom game instance and starts a new episode. 
game.init()

In [3]:
# creating a matrix identity, its mean, same number of columns and rows
actions = np.identity(7, dtype=np.uint8)

In [4]:
# Returns GameState object with the current game state. (The Episode)
state = game.get_state()

In [5]:
# return some game variables, for exemple: the health.
state.game_variables

array([100.])

In [6]:
# number of episodes
num_episodes = 10

for episode in range(num_episodes):
    # Initializes a new episode. The state of an environment is completely restarted (all variables and rewards are reset to their initial values). After calling this method, the first state from the new episode will be available. 
    game.new_episode()

    while not game.is_episode_finished():
        # again, get the state
        state = game.get_state()

        # Get the game image 
        img = state.screen_buffer

        # get some data from the game
        info = state.game_variables

        # make random decision
        decision = random.choice(actions)

        # do an decision
        reward = game.make_action(decision, 4)

        # await
        time.sleep(0.02)
    
    print('Result:', game.get_total_reward())
    time.sleep(2)

Result: -115.08735656738281
Result: -88.1358642578125
Result: -53.03684997558594
Result: -115.97743225097656
Result: -76.27359008789062
Result: -114.78858947753906
Result: -89.60504150390625
Result: -33.67433166503906
Result: -76.70585632324219
Result: -85.45492553710938


In [7]:
# finish the game
game.close()

Converting it to a Gym Environment

In [26]:
# Import environment base class from OpenAI Gym - https://gymnasium.farama.org/
from gym import Env
# Import gym spaces 
from gym.spaces import Discrete, Box
# Import opencv - https://opencv.org/ - https://pypi.org/project/opencv-python/
import cv2


In [27]:

# Create Vizdoom OpenAI Gym Environment
class VizDoomGym(Env):  
    # Function that is called when we start the env
    def __init__(self, render=False, config='resource/ViZDoom/scenarios/deadly_corridor.cfg'): 
        # Inherit from Env
        super().__init__()
        # Setup the game 
        self.game = vd.DoomGame()
        self.game.load_config(config)
        
        # Render frame logic
        if render == False: 
            self.game.set_window_visible(False)
        else:
            self.game.set_window_visible(True)
        
        # Start the game 
        self.game.init()
        
        # Create the action space and observation space
        self.observation_space = Box(low=0, high=255, shape=(100,160,1), dtype=np.uint8) 
        self.action_space = Discrete(7)
        
        # Game variables: HEALTH DAMAGE_TAKEN HITCOUNT SELECTED_WEAPON_AMMO
        self.damage_taken = 0
        self.hitcount = 0
        self.ammo = 52 ## CHANGED
        
        
    # This is how we take a step in the environment
    def step(self, action):
        # Specify action and take step 
        actions = np.identity(7)
        movement_reward = self.game.make_action(actions[action], 4) 
        
        reward = 0 
        # Get all the other stuff we need to retun 
        if self.game.get_state(): 
            state = self.game.get_state().screen_buffer
            state = self.grayscale(state)
            
            # Reward shaping
            game_variables = self.game.get_state().game_variables
            health, damage_taken, hitcount, ammo = game_variables
            
            # Calculate reward deltas
            damage_taken_delta = -damage_taken + self.damage_taken
            self.damage_taken = damage_taken
            hitcount_delta = hitcount - self.hitcount
            self.hitcount = hitcount
            ammo_delta = ammo - self.ammo
            self.ammo = ammo
            
            reward = movement_reward + damage_taken_delta*10 + hitcount_delta*200  + ammo_delta*5 
            info = ammo
        else: 
            state = np.zeros(self.observation_space.shape)
            info = 0 
        
        info = {"info":info}
        done = self.game.is_episode_finished()
        
        return state, reward, done, info 
    
    # Define how to render the game or environment 
    def render(): 
        pass
    
    # What happens when we start a new game 
    def reset(self): 
        self.game.new_episode()
        state = self.game.get_state().screen_buffer
        return self.grayscale(state)
    
    # Grayscale the game frame and resize it 
    def grayscale(self, observation):
        gray = cv2.cvtColor(np.moveaxis(observation, 0, -1), cv2.COLOR_BGR2GRAY)
        resize = cv2.resize(gray, (160,100), interpolation=cv2.INTER_CUBIC)
        state = np.reshape(resize, (100,160,1))
        return state
    
    # Call to close down the game
    def close(self): 
        self.game.close()

In [10]:
#env.close()

In [28]:
env = VizDoomGym(render=True)

In [21]:
state = env.reset()

In [22]:
env.reset()

array([[[32],
        [33],
        [25],
        ...,
        [27],
        [23],
        [24]],

       [[27],
        [33],
        [23],
        ...,
        [24],
        [24],
        [24]],

       [[20],
        [35],
        [23],
        ...,
        [24],
        [24],
        [24]],

       ...,

       [[75],
        [63],
        [62],
        ...,
        [44],
        [71],
        [60]],

       [[15],
        [48],
        [47],
        ...,
        [49],
        [69],
        [47]],

       [[22],
        [14],
        [26],
        ...,
        [57],
        [37],
        [39]]], shape=(100, 160, 1), dtype=uint8)

In [14]:

# Import Environment checker - https://stable-baselines3.readthedocs.io/en/master/
from stable_baselines3.common import env_checker

from matplotlib import pyplot as plt

In [32]:
env_checker.check_env(env)

plt.imshow(cv2.cvtColor(state, cv2.COLOR_BGR2RGB))

AssertionError: Your environment must inherit from the gymnasium.Env class cf. https://gymnasium.farama.org/api/env/